In [1]:
# Importing Libraries 
import numpy as np
import pandas as pd

# Vusualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns

#Import Train Test data
import random
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

#Avoid warnings
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
#Importing Data
loan  = pd.read_csv("https://raw.githubusercontent.com/amitbcp/machine_learning_problems/master/case_studies/lending_club/data/loan.csv" )

In [ ]:
loan.head()

In [ ]:
loan.shape

We have around 40K rows and 111 Columns

In [ ]:
loan.describe()

In [ ]:
#Check the percentage of missing data in each column

In [ ]:
loan.isnull().mean()

In [ ]:
# Clone the entire repo.
#!git clone -l -s git://github.com/jakevdp/PythonDataScienceHandbook.git cloned-repo
#%cd cloned-repo
#!ls

In [ ]:
# Fetch a single <1MB file using the raw GitHub URL.
#!curl --remote-name \
#    -H 'Accept: application/vnd.github.v3.raw' \
#     --location https://api.github.com/repos/jakevdp/PythonDataScienceHandbook/contents/notebooks/data/california_cities.csv

In [ ]:
missing_column = 100*(loan.isnull().sum()/len(loan.id))

In [ ]:
missing_column.loc[missing_column > 50]

mths_since_last_delinq             64.662487
mths_since_last_record             92.985372
next_pymnt_d                       97.129693
mths_since_last_major_derog       100.000000
annual_inc_joint                  100.000000
dti_joint                         100.000000
verification_status_joint         100.000000
tot_coll_amt                      100.000000
tot_cur_bal                       100.000000
open_acc_6m                       100.000000
open_il_6m                        100.000000
open_il_12m                       100.000000
open_il_24m                       100.000000
mths_since_rcnt_il                100.000000
total_bal_il                      100.000000
il_util                           100.000000
open_rv_12m                       100.000000
open_rv_24m                       100.000000
max_bal_bc                        100.000000
all_util                          100.000000
total_rev_hi_lim                  100.000000
inq_fi                            100.000000
total_cu_t

In [ ]:
print("TOtal column with more than 50% data missing :", len(missing_column.loc[missing_column > 50]))

TOtal column with more than 50% data missing : 57


In [ ]:
print("columns which will be dropped : ", missing_column.loc[missing_column > 50].index)

columns which will be dropped :  Index(['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d',
       'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'tot_coll_amt', 'tot_cur_bal',
       'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
       'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m',
       'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi',
       'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal',
       'bc_open_to_buy', 'bc_util', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 

In [ ]:
drop_columns = missing_column.loc[missing_column > 50].index

Droping Columns with greater than 50% missing data

Droping Columns with greater than 50% missing data

In [ ]:
# dropping these columns
print("shape before dropping",loan.shape)
loan = loan.drop(drop_columns,axis=1)
print("shape after dropping",loan.shape)

shape before dropping (39717, 111)
shape after dropping (39717, 54)


In [ ]:
missing_column = 100*(loan.isnull().sum()/len(loan.id)) #Checking Null Values

In [ ]:
missing_column.loc[missing_column > 0]

emp_title                      6.191303
emp_length                     2.706650
desc                          32.580507
title                          0.027696
revol_util                     0.125891
last_pymnt_d                   0.178765
last_credit_pull_d             0.005036
collections_12_mths_ex_med     0.140998
chargeoff_within_12_mths       0.140998
pub_rec_bankruptcies           1.754916
tax_liens                      0.098195
dtype: float64

In [ ]:
#Let's remove desc as well as it is not giving us useful information on loan predcition
loan = loan.drop('desc',axis=1)

In [ ]:
#let's identify unique values in data set

loan_column = loan.columns

for col in loan_column:
  print("unique entries in " + col + ": %d" %len(loan[col].unique()))
  if (len(loan[col].unique())) < 20:
    print(loan[col].unique())
    print("\n") 

unique entries in id: 39717
unique entries in member_id: 39717
unique entries in loan_amnt: 885
unique entries in funded_amnt: 1041
unique entries in funded_amnt_inv: 8205
unique entries in term: 2
[' 36 months' ' 60 months']


unique entries in int_rate: 371
unique entries in installment: 15383
unique entries in grade: 7
['B' 'C' 'A' 'E' 'F' 'D' 'G']


unique entries in sub_grade: 35
unique entries in emp_title: 28821
unique entries in emp_length: 12
['10+ years' '< 1 year' '1 year' '3 years' '8 years' '9 years' '4 years'
 '5 years' '6 years' '2 years' '7 years' nan]


unique entries in home_ownership: 5
['RENT' 'OWN' 'MORTGAGE' 'OTHER' 'NONE']


unique entries in annual_inc: 5318
unique entries in verification_status: 3
['Verified' 'Source Verified' 'Not Verified']


unique entries in issue_d: 55
unique entries in loan_status: 3
['Fully Paid' 'Charged Off' 'Current']


unique entries in pymnt_plan: 1
['n']


unique entries in url: 39717
unique entries in purpose: 14
['credit_card' 'c

In [ ]:
Columns_not_useful = ['id','member_id','pymnt_plan','zip_code','initial_list_status','collections_12_mths_ex_med','policy_code','application_type','acc_now_delinq',
                      'chargeoff_within_12_mths','delinq_amnt','tax_liens']

In [ ]:
print('Shape before dropping not useful columns', loan.shape)
loan = loan.drop(Columns_not_useful,axis=1)
print('Shape after removing not useful columns', loan.shape )

Shape before dropping not useful columns (39717, 53)
Shape after removing not useful columns (39717, 41)


In [ ]:
# Remove the following Columns which does not provide any meaning to EDA analysis. 
print('Dropping the following list of columns as these columns are not needed for the EDA analysis')

column_list = ['id','member_id','funded_amnt','pymnt_plan','url','zip_code','initial_list_status','policy_code'
               ,'application_type','acc_now_delinq','delinq_amnt']

for column in column_list:
    print('Dropping', column, '...')
    try:
        loan.drop(column, axis=1, inplace=True)
    except:
        print('\tColumn already dropped...')
        continue

Dropping the following list of columns as these columns are not needed for the EDA analysis
Dropping id ...
	Column already dropped...
Dropping member_id ...
	Column already dropped...
Dropping funded_amnt ...
Dropping pymnt_plan ...
	Column already dropped...
Dropping url ...
Dropping zip_code ...
	Column already dropped...
Dropping initial_list_status ...
	Column already dropped...
Dropping policy_code ...
	Column already dropped...
Dropping application_type ...
	Column already dropped...
Dropping acc_now_delinq ...
	Column already dropped...
Dropping delinq_amnt ...
	Column already dropped...


In [ ]:
# Remove the following Columns are not always present at the time of loan application.

print('Dropping the following list of columns as these columns are not needed for the EDA analysis')

column_list = [
    "delinq_2yrs","earliest_cr_line","inq_last_6mths","out_prncp","out_prncp_inv","total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee",
    "application_type","last_pymnt_d","last_credit_pull_d"]


for column in column_list:
    print('Dropping', column, '...')
    try:
        loan.drop(column, axis=1, inplace=True)
    except:
        print('\tColumn already dropped...')
        continue

Dropping the following list of columns as these columns are not needed for the EDA analysis
Dropping delinq_2yrs ...
Dropping earliest_cr_line ...
Dropping inq_last_6mths ...
Dropping out_prncp ...
Dropping out_prncp_inv ...
Dropping total_pymnt ...
Dropping total_pymnt_inv ...
Dropping total_rec_prncp ...
Dropping total_rec_int ...
Dropping total_rec_late_fee ...
Dropping recoveries ...
Dropping collection_recovery_fee ...
Dropping application_type ...
	Column already dropped...
Dropping last_pymnt_d ...
Dropping last_credit_pull_d ...


In [ ]:
loan.nunique().sort_values()

term                        2
pub_rec_bankruptcies        3
loan_status                 3
verification_status         3
pub_rec                     5
home_ownership              5
grade                       7
emp_length                 11
purpose                    14
sub_grade                  35
open_acc                   40
addr_state                 50
issue_d                    55
total_acc                  82
int_rate                  371
loan_amnt                 885
revol_util               1089
dti                      2868
annual_inc               5318
funded_amnt_inv          8205
installment             15383
title                   19615
revol_bal               21711
emp_title               28820
last_pymnt_amnt         34930
dtype: int64

In [ ]:
loan.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,addr_state,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_pymnt_amnt,pub_rec_bankruptcies
0,5000,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,credit_card,Computer,AZ,27.65,3,0,13648,83.70%,9,171.62,0.0
1,2500,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,car,bike,GA,1.00,3,0,1687,9.40%,4,119.66,0.0
2,2400,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,small_business,real estate business,IL,8.72,2,0,2956,98.50%,10,649.91,0.0
3,10000,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,other,personel,CA,20.00,10,0,5598,21%,37,357.48,0.0
4,3000,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-11,Current,other,Personal,OR,17.94,15,0,27783,53.90%,38,67.79,0.0


Data Formatting

Converting Date time column

In [ ]:
datetime_Column = ['issue_d']

for col in datetime_Column:
  loan[col] = pd.to_datetime(loan[col],format='%b-%y')

#Extracting month and year from issue date
loan['issue_month'] = loan['issue_d'].apply(lambda x: x.month)
loan['issue_year'] = loan['issue_d'].apply(lambda x: x.year)


In [ ]:
loan.groupby('issue_year').issue_year.count()

issue_year
2007      251
2008     1562
2009     4716
2010    11532
2011    21656
Name: issue_year, dtype: int64

In [ ]:
loan['int_rate'] = loan['int_rate'].str.strip('%').astype('float')

In [ ]:
# Let's check Employee Length Columns
#In Data dict this field represent number of yesrs of experience 
# Let's try to set this in experience hirarchey where in 0 is less than one year and 10 is more than 10+ experience 

loan.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [ ]:
loan.isnull().mean()

loan_amnt               0.000000
funded_amnt_inv         0.000000
term                    0.000000
int_rate                0.000000
installment             0.000000
grade                   0.000000
sub_grade               0.000000
emp_title               0.061913
emp_length              0.027066
home_ownership          0.000000
annual_inc              0.000000
verification_status     0.000000
issue_d                 0.000000
loan_status             0.000000
purpose                 0.000000
title                   0.000277
addr_state              0.000000
dti                     0.000000
open_acc                0.000000
pub_rec                 0.000000
revol_bal               0.000000
revol_util              0.001259
total_acc               0.000000
last_pymnt_amnt         0.000000
pub_rec_bankruptcies    0.017549
issue_month             0.000000
issue_year              0.000000
dtype: float64

In [ ]:
emp_length_dict = {
    '< 1 year' : 0,
    '1 year'  : 1, 
    '2 years'  : 2,
    '3 years'  : 3,
    '4 years'  : 4,
    '5 years'  : 5,
    '6 years'  : 6,
    '7 years'  : 7,
    '8 years'  : 8,
    '9 years'  : 9,
    '10+ years' : 10
    }

loan = loan.replace({"emp_length": emp_length_dict})

In [ ]:
#let's work on term value
print(loan.term.value_counts())

 36 months    29096
 60 months    10621
Name: term, dtype: int64


In [ ]:
loan['term'] = loan.term.apply(lambda x : x.split()[0])

In [ ]:
print(loan.term.value_counts())

36    29096
60    10621
Name: term, dtype: int64


In [ ]:
loan.to_csv('/drive/My Drive/Colab Notebooks/Resume Project/Lending Club/LOAN_CLEANED.csv', encoding= 'ISO-8859-1', index=False)


In [ ]:
loan_cleaned = pd.read_csv('/drive/My Drive/Colab Notebooks/Resume Project/Lending Club/LOAN_CLEANED.csv', encoding = 'ISO-8859-1',low_memory=False)

In [ ]:
# Filtering Full Pay and Charge off
loan_cleaned = loan_cleaned[loan_cleaned['loan_status']!='Current']
loan_cleaned['Defaulted'] = loan_cleaned['loan_status'].apply(lambda x : 'No' if x=='Fully Paid' else 'Yes')
loan_cleaned['loan_status']= loan_cleaned['loan_status'].apply(lambda x : 0 if x== 'Fully Paid' else 1)

# Converting Loan status into integer type
loan_cleaned['loan_status'] = loan_cleaned['loan_status'].apply(lambda x: pd.to_numeric(x))

In [ ]:
#list of variable we have available for analysis.
loan_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38577 entries, 0 to 39716
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   loan_amnt             38577 non-null  int64  
 1   funded_amnt_inv       38577 non-null  float64
 2   term                  38577 non-null  int64  
 3   int_rate              38577 non-null  float64
 4   installment           38577 non-null  float64
 5   grade                 38577 non-null  object 
 6   sub_grade             38577 non-null  object 
 7   emp_title             36191 non-null  object 
 8   emp_length            37544 non-null  float64
 9   home_ownership        38577 non-null  object 
 10  annual_inc            38577 non-null  float64
 11  verification_status   38577 non-null  object 
 12  issue_d               38577 non-null  object 
 13  loan_status           38577 non-null  int64  
 14  purpose               38577 non-null  object 
 15  title              

In [ ]:
nominal = ['term','emp_length','grade','sub_grade','home_ownership','verification_status','purpose']
Quant = ['loan_amnt','int_rate','installment','annual_inc','dti']
Target = ['loan_status']

Univariate Analysis

In [ ]:
def plot_cat(cat_var,title ="Nominal variable plot",ylabel="No. of loans"):
  plt.figure(figsize=(8,6))
  order = loan_cleaned[cat_var].value_counts(ascending=True).index
  ax = sns.countplot(y=cat_var, data= loan_cleaned, order=order, palette='muted')
  ax.set_title(title)
  ax.set_xlabel(ylabel)
  plt.show()

def plot_quant(quant_var,title="Quantitative variable plot",ylabel="fraction of loans"):
    #     sns.barplot(x=cat_var, y='loan_status', data=loan_cleaned)
    plt.figure(figsize=(8, 6))
    #   sns.distplot(loan_cleaned[quant_var])
    
    ax = sns.distplot(loan_cleaned[quant_var],color='b')
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax2 = ax.twinx()
    sns.boxplot(x=loan_cleaned[quant_var], ax=ax2)
    ax2.set(ylim=(-0.5, 10))
    plt.show()

 # lets define a function to plot loan_status across categorical variables
def plot_seg_cat(cat_var):
    order = loan_cleaned[cat_var].value_counts(ascending=True).index
    sns.barplot(x=cat_var, y='loan_status', data=loan_cleaned)
    plt.show()

 # lets define a function to plot loan_status across categorical variables
def plot_seg_univariate(x_cat_var, y_quant, title="Quantitative variable plot",xlabel="fraction of loans",ylabel="fraction of loans"):
    #plt.figure(figsize=(8, 6))
    order = sorted(set(loan_cleaned[x_cat_var]))
    if x_cat_var == 'home_ownership':
        df = loan_cleaned[loan_cleaned[x_cat_var]!='NONE']
        ax = sns.barplot(x=x_cat_var, y=y_quant, data = df, palette = 'muted')
    else:
        ax = sns.barplot(x=x_cat_var, y=y_quant, data=loan_cleaned, palette = 'muted', order = order)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    #plt.show()

1.1 Target variable : loan_status

In [ ]:
loan_cleaned['loan_status'].value_counts()

Target Variable is not in proportion that may lead to bias modelling so We need to handle this data imbalance. 

In [ ]:
plt.figure(figsize=(8, 6))
ax = sns.barplot(x="Defaulted", y="loan_status", hue='Defaulted',data=loan_cleaned, estimator=lambda x: len(x) / len(loan_cleaned) * 100)
ax.set(ylabel="Percent")
ax.set(xlabel="defaulted")
ax.set_title("Distribution of Loans")

Let's analyse Categorical fields

In [ ]:
#Term
plot_cat('term',"Distribution of loans over term lenghth(in months)")


In [ ]:
plot_cat('emp_length',"Distribution of loans over term lenghth(in months)")


- We have capped employee experience to 10 so we can see there are most loan cases with 10+ years of experience. 

- People with less saving require financial loan as support so less experience more the loan demand

- Trend of requirement of loan is declining as experience grows 


# Grade

In [ ]:
plot_cat('grade',"Distribution of loans over term lenghth(in months)")


- Grade B has most loans





# Sub-Grade



*   No of loan decreases as grade decreases 




In [ ]:
title = "Distribution of loan over sub Grade"
ylabel = "No. of loans"
cat_var = "sub_grade"
plt.figure(figsize=(12,6))
order = sorted(set(loan_cleaned[cat_var]))
ax= sns.countplot(x=cat_var,data=loan_cleaned,order=order,palette='muted')
ax.set_title(title)
ax.set_ylabel(ylabel)
ax.set_xlabel("sub-grade")
plt.show()

## Home Ownership

People who live on rent or have mortgage applied for loan. we need to study this well because it will increase the risk for being defaulter.

In [ ]:
plot_cat('home_ownership',"Distribution of loans over term lenghth(in months)")


## Purpose 

Debt consolidation and Credit card are two important reasons for lending money. This also may increase the risk of defaulters as it shows bad finance management on user end. 

Mostly Credit card charges higher interest so people can lend money with lower interest and pay off their Credit card bills to save additional interest. 

In [ ]:
plot_cat('purpose',"Distribution of loans over term lenghth(in months)")


# Quantitative Variable 


Let's Analse 'loan_amnt','int_rate','installment','annual_inc','dti'

## Loan Amount 

- Prople generally takes loan in the range of 5K - 15K

In [ ]:
plot_quant('loan_amnt',"Distribution of Loan amount")


Int Rate

- Most loan available in the range of 8 to 15% 
- Credit card charges hiher than this that why purpose of taking loan as credit card is higher. 

In [ ]:
plot_quant('int_rate',"Distribution of Loan amount")


In [ ]:
plot_quant('installment',"Distribution of Loan amount")


In [ ]:
plot_quant('annual_inc',"Distribution of Loan amount")



In [ ]:
plot_quant('dti',"Distribution of Dti")


# Segmented Univariate Analysis:

Let us start with the segmented univariate analysis where quantitative variables distribution will be segmented across categorical variable. we are going to consider Grade, Home Ownership, Term, and Emp Length as our segments

## 2.1. Loan Status

In [ ]:
plt.rcdefaults()
plt.figure(figsize=(16, 10))

plt.subplot(2, 2, 1)
plot_seg_univariate('grade','loan_status',"Avg. default rate across Grade",'grade',"default rate")

plt.subplot(2, 2, 2)
plot_seg_univariate('home_ownership','loan_status',"Avg. default rate across Home ownership",'home_ownership',"default rate")

plt.subplot(2, 2, 3)
plot_seg_univariate('term','loan_status',"Avg. default rate across term",'term',"default rate")

plt.subplot(2, 2, 4)
plot_seg_univariate('emp_length','loan_status',"Avg. default rate across Emp Exp",'emp_length',"default rate")

plt.show()



*   Default rate increase as grade decrease
*   Term Duration has substantial difference on default rate. 60 months term have highest default rate. 



In [ ]:
#loan_cleaned['emp_length'] = loan_cleaned['emp_length'].replace({"10+ years" : 10})

In [ ]:
#loan_cleaned['emp_length'] = loan_cleaned['emp_length'].astype(int)

## Loan Amount

In [ ]:
plt.rcdefaults()
plt.figure(figsize=(16, 10))

plt.subplot(2, 2, 1)
plot_seg_univariate('grade','loan_amnt',"Avg. loan amount across Grade",'grade',"loan_amnt")

plt.subplot(2, 2, 2)
plot_seg_univariate('home_ownership','loan_amnt',"loan amount across Home ownership",'home_ownership',"loan_amnt")

plt.subplot(2, 2, 3)
plot_seg_univariate('term','loan_amnt',"loan amount across term",'term',"loan_amnt")

plt.subplot(2, 2, 4)
plot_seg_univariate('emp_length','loan_amnt',"Loan amount across Emp Exp",'emp_length',"loan_amnt")

plt.show()



1.   Loan amount increase as grade decrease
2.   60 months term have almost 1.5 time of 36 months term loan amount
3.   People without Mortgage has highest loan amount  



## Interest Rate

In [ ]:
plt.rcdefaults()
plt.figure(figsize=(16, 10))

plt.subplot(2, 2, 1)
plot_seg_univariate('grade','int_rate',"Interest rate across Grade",'grade',"int_rate")

plt.subplot(2, 2, 2)
plot_seg_univariate('home_ownership','int_rate',"Interest rate across Home ownership",'home_ownership',"int_rate")

plt.subplot(2, 2, 3)
plot_seg_univariate('term','int_rate',"Interest rate across term",'term',"int_rate")

plt.subplot(2, 2, 4)
plot_seg_univariate('emp_length','int_rate',"Interest rate across Emp Exp",'emp_length',"int_rate")

plt.show()



*   Interest rate is higher as grade decreases
*   Interest rate is higher if the term is more. 



## Installment 

In [ ]:
plt.rcdefaults()
plt.figure(figsize=(16, 10))

plt.subplot(2, 2, 1)
plot_seg_univariate('grade','installment',"Installment across Grade",'grade',"installment")

plt.subplot(2, 2, 2)
plot_seg_univariate('home_ownership','installment',"Installment  across Home ownership",'home_ownership',"installment")

plt.subplot(2, 2, 3)
plot_seg_univariate('term','installment',"Installment  across term",'term',"installment")

plt.subplot(2, 2, 4)
plot_seg_univariate('emp_length','installment',"Installment  across Emp Exp",'emp_length',"installment")

plt.show()

  
*   We can see the Installment increasing as the Grade decreasing.
*   60 month term has a higher installment than of the 36 month term.



## Purpose of the Loan

In [ ]:
plt.rcdefaults()
plt.figure(figsize=(22, 15))

plt.subplot(3,1, 1)
plot_seg_univariate('purpose','loan_status',"Avg. default rate across Purpose",'purpose',"loan status")

plt.subplot(3,1, 2)
plot_seg_univariate('purpose','loan_amnt',"Avg. loan amount across Purpose",'purpose',"loan amount")

plt.subplot(3,1, 3)
plot_seg_univariate('purpose','int_rate',"Avg. interest rate across Purpose",'purpose',"interest rate")

plt.show()



*   small business have the highest default rate.
* Wedding are purpose with the lowest default rate.
* Debt consolidation loans default rate are in the middle.




# Metrics

### Business Metrics
We wanted to understand which loans may be defaulted. Bases on the provided business rules we create new column "Defaulted"

In [ ]:
loan_cleaned["Defaulted"] = loan_cleaned["loan_status"].apply(lambda x: 'yes' if x==1 else 'No')

In [ ]:
print('Number of Loans Deafulted after merging Current and Fully Paid as Not Defaulted \n',
      loan_cleaned.groupby('Defaulted')['Defaulted'].count())

### 3.2. Data Metrics :

The risk of missing EMI's depend on a person monthly exposure towards loan as a function of it's monthly income. We derive the percentage of monthly income exposed towards payment of loan.

In [ ]:
loan_cleaned['instal_income_perc'] = ((loan_cleaned['installment'])/(loan_cleaned['annual_inc']/12))*100
loan_cleaned['instal_income_perc'].describe()

* We can see that monthly exposure of income towards EMI varies from as low as 0.02% to 32%
* This also showcases the skewness in income which was previously established.
* This metrics can directly be used as a risk calculation factor.





# Bivariate Analysis

In [ ]:
# Replotting a Focused heatmap based on the Above Pandas profiler
plt.figure(figsize=(20,16))
cont_var= ['loan_amnt','funded_amnt_inv','installment','annual_inc','pub_rec','pub_rec_bankruptcies','dti',
           'emp_length','home_ownership','revol_bal', 'revol_util','open_acc','total_acc',
           'Defaulted']

corr = loan_cleaned[cont_var].corr()
sns.heatmap(corr, annot=True, center=0.5)

From the Co-relation map, the following can be intepreted :

- High +ve Co-relation : loan_amnt, funded_amnt_inv,installment
- Medium +ve Co-relation : annual_inc with each of the (loan_amnt, funded_amnt_inv,installment)
- High +ve Co-relation : pub_rec & pub_rec_bankruptcies
- Medium -ve Co-relation : dti & annual_inc

In [ ]:
loan_cleaned.head()

In [ ]:
sns.heatmap(loan_cleaned.isnull()*100)

In [ ]:
loan_cleaned.isnull().mean()

In [ ]:
#loan_cleaned.loc[(loan_cleaned['emp_length']==0) & (loan_cleaned['emp_title'].isnull())].replace(to_replace = np.nan, value= 'NoExperience',inplace = True)

In [ ]:
loan_cleaned[loan_cleaned['emp_length']== 0].groupby('emp_title').sum()

In [ ]:
loan_cleaned[(loan_cleaned['emp_length']==0) & (loan_cleaned['emp_title'].isnull())]['emp_title']

In [ ]:
loan_cleaned[(loan_cleaned['emp_length']==0) & (loan_cleaned['emp_title'].isnull())]['emp_title']

In [ ]:
loan_cleaned['emp_title'].fillna('missing',inplace = True)

In [ ]:
loan_cleaned['emp_title'].isnull().sum()

In [ ]:
#loan_cleaned['emp_title'].value_counts().sort_values(ascending=False).plot.bar()


In [ ]:
def Replace_Mean(var, dataset):
  mean = dataset[var].mean()
  return dataset[var].fillna(mean)



In [ ]:
Replace_Mean('emp_length',loan_cleaned)

In [ ]:
#loan_cleaned['emp_length']= loan_cleaned['emp_length'].astype(int)

In [ ]:
loan_cleaned['emp_length'].isnull().sum()

In [ ]:
loan_cleaned['emp_length'] = loan_cleaned['emp_length'].fillna(method='ffill')

In [ ]:
loan_cleaned['emp_length'].isnull().sum()

In [ ]:
loan_cleaned['emp_length']= loan_cleaned['emp_length'].astype(int)

In [ ]:
loan_cleaned['title'].head()

In [ ]:
#= loan_cleaned['title'].fillna('Personel', inplace = True)

In [ ]:
loan_cleaned['title'].isnull().sum()

In [ ]:
loan_cleaned['title'].fillna('Personal', inplace = True)

In [ ]:
loan_cleaned['title'].isnull().sum()

In [ ]:
loan_cleaned['revol_util'] = loan_cleaned['revol_util'].fillna(method='bfill')

In [ ]:
loan_cleaned['revol_util'].isnull().sum()

In [ ]:
loan_cleaned['pub_rec_bankruptcies'].isnull().sum()

In [ ]:
loan_cleaned['pub_rec_bankruptcies'] = loan_cleaned['pub_rec_bankruptcies'].fillna(value = 0.0)

In [ ]:
loan_cleaned['pub_rec_bankruptcies'].value_counts()

In [ ]:
loan_cleaned.info()

In [ ]:
#Encoding = ['grade','home_ownership','purpose']
#Remove = ['sub_grade','emp_title','verification_status','issue_d','title','addr_state','revol_util','grade','home_ownership','purpose']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
loan_cleaned.info()

In [ ]:
tmp = pd.get_dummies(loan_cleaned[['grade','home_ownership','purpose']], drop_first=True)



In [ ]:
#loan_cleaned= pd.concat([loan_cleaned,tmp], axis=1).head()

In [ ]:
#loan_cleaned.head(20)

In [ ]:
tmp

In [ ]:
print(tmp.shape)
print(loan_cleaned.shape)

In [ ]:
loan_cleaned = pd.concat([loan_cleaned, tmp], axis=1)


In [ ]:
loan_cleaned.shape

In [ ]:
loan_cleaned = loan_cleaned.drop(['sub_grade','emp_title','verification_status','issue_d','title','addr_state','revol_util','grade','home_ownership','purpose'], axis=1)

In [ ]:
loan_cleaned.info()

In [ ]:
loan_cleaned['Defaulted'] = loan_cleaned['Defaulted'].apply(lambda x: 1 if x=='yes' else 0 )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn import *

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    loan_cleaned.drop('Defaulted', axis=1),  # predictors
    loan_cleaned['Defaulted'],  # target
    test_size=0.1,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
print(y_pred)
print("\n")

In [ ]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test) ## y_pred=Vector of Predictions
print('y_pred=',y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print('Confusion Matrix:',cm)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))